In [17]:
# edited Xt0, Xt1, change some for loops to vectorization for efficiency, edited tradeSharest0, error is arising from edited Xt1

#add alphaj, edit Xt0 (equation (8)) and Xt1 equations (and replace incomet0 and t1), create tradesharest0 function from equation (7)

#2/28/2025 with d1wdot instead of wdot
using Random, Distributions, Optim, NLsolve, SpecialFunctions
N = 2 # Number of countries
periods = 7
J = 3 # Number of goods (needs to be big number and an integer type)
θ = fill(4.0, J) # Frechet shape parameter (governs comparative advantage)
T = ones(N*J, periods) # Frechet scale parameter (governs absolute advantage)

v = 1.0 # migration elasticity
β = .99 # discount factor


σ = 2 # Substitution elasticity between goods
τ = ones(N*J, N, periods) # Iceberg trade costs
L = ones(N*J, periods) # Size of labor force in each country
Ldot = ones(N*J, periods)
α = ones(J) * (1/J) # final good expenditure share

At0 = ones(N*J) #initial productivities
wt = ones(N*J, periods)
wt0 = ones(N*J)
Lt = ones(N*J, periods)

wdot = ones(N*J, periods)
tradesharest0 = ones(N*J, N) * (1 / N)
Adot = ones(N*J, periods)

kdot = ones(N*J, N, periods)
TSt1 = ones(N*J, N, periods) 
pdotArray = ones(N*J, periods)
d1wdot = ones(N*J)
μ = zeros(N*J, N*J, periods)

μtminus1 = ones(N*J, N*J) #value of μ at time t = -1
πt0 = ones(N*J, N) #initial trade shares
wLt0 = ones(N*J, N) #initial values of w*L
rHt0 = ones(N*J, N) #initial values of r*H
μt = zeros(N*J,N*J,periods)

# a guess for path of udot:
udotPathGuess = ones(N*J, periods)
udotPathUpdate = zeros(N*J, periods)
errormax = 1.0 #the maximum log difference between guesses and updates for appendix D algorithm

function θindex(j)
    div(j-1, N) + 1  # Find the correct index for θ
end

function pdot(n, d1wdot, kdot, Adot, time)
    j = θindex(n)
    (sum(tradesharest0[n, i] * (d1wdot[(j-1) * N + i] * kdot[n, i, time])^-θ[j] * Adot[(j-1) * N + i, time]^θ[j] for i in 1:N))^(-1 / θ[j])
end ##### Going to need to update tradeshares

#=function tradeSharest0(n, j, i, wt0, τ, At0) # total expenditure in market (n, j) on goods j from market i
    njindex = ((n - 1) * j) + j #index for nj arrays
    ijindex = ((i - 1) * j) + j #index for ij arrays
    ((wt0[ijindex] * τ[njindex, ijindex]) ^ -θ[j] * At0[ijindex] ^ -θ[j]) / (sum((wt0[((m - 1) * j) + j)] * τ[((m - 1) * j) + j]) ^ -θ[j] * At0[((m - 1) * j) + j] ^ θ[j] for m in 1:J) #from equation (7)
end =# ## I think this might be incorrect

function tradeSharest0(n, i, wt0, At0)
    j = θindex(n)
    wt0[(j-1) * N + i] * τ[n,i]^-θ[j] * At0[(j-1) * N + i] ^θ[j] / (sum((wt0[(j-1) * N + m] *τ[n,m])^-θ[j] * At0[(j-1) * N + m]^θ[j] for m in 1:N))
end

function tradeSharest1(n, i, d1wdot, Ldot, Adot, kdot, time)
    j = θindex(n)
    tradesharest0[n, i] * ((d1wdot[(j-1) * N + i] * kdot[n, i, time]) / pdot(n, d1wdot, kdot, Adot, time))^-θ[j] * Adot[(j-1) * N + i, time]^θ[j]
end

function incomet1(n, d1wdot, time)
    wt[n, time]*(d1wdot[n])*L[n, time]*(Ldot[n, time])
end

#=
function Xt1(n, d1wdot, tradesharest0, Ldot, Adot, kdot, time)
    sum(incomet1(n, d1wdot, time) * tradeSharest1(n, i, d1wdot, Ldot, Adot, kdot, time) for i in 1:N)
end
=#
### Implimentation of fixed Xt1 results in error


function Xt1(n, α, d1wdot, Lt, wt, Ldot, time)
    j = θindex(n)
    country_index = mod(n - 1, N) + 1
    α[j] * sum(d1wdot[country_index + N*(k-1)] * Ldot[country_index + N*(k-1), time] * wt[country_index + N*(k-1), time] * Lt[country_index + N*(k-1), time] for k in 1:J) #from equation (14) 
end


while errormax > .01 
    for time in 1:periods-1
        for n in 1:N*J
            for i in 1:N*J
                μt[n,i,time+1] = (μt[n,i,time]*(udotPathGuess[i,time+1])^(β/v))/sum(μt[n,h,time]*(udotPathGuess[h,time+1])^(β/v) for h in 1:N*J)
            end
        end
        Lt[:, time + 1] .+= μt[:, :, time] * Lt[:, time] ### Step 3 of the appendix D

    end
    ################### solving for wt0
    function incomet0(n, wt0, Lt) #income of country-sector in time t=0 given wage and labor  
        wt0[n] * Lt[n, 1] 
    end
    
    #=
    function Xt0(n, j, α, wt0, Lt) #expenditure on sector good j in region n
        k = 1 + ((n - 1) * j) #index for the beginning of the range of region n's sectors 
        α[j] * sum(wt0[k + i] * Lt[k + i, 1] for i in 0:(J-1)) #from equation (8)
    end
    =#

    function Xt0(n, α, wt0, Lt) #expenditure on sector good j in region n
        j = θindex(n)
        country_index = mod(n - 1, N) + 1
        α[j] * sum(wt0[country_index + N*(k-1)] * Lt[country_index + N*(k-1), 1] for k in 1:J) #from equation (8)
    end

    function g!(G, wt0)
        for n in 1:N*J
            j = div(n-1, N) + 1  # Find the correct index for j
            country = n % N # finding correct index for country
            if country == 0
                country = N
            end
            #################FIXME
            G[n] = incomet0(n, wt0, Lt) - sum(incomet0((j-1) * N + i, wt0, Lt) * tradesharest0[(j-1) * N + i, country] for i in 1:N) #market clearing for wt0
        end
    end
    results = nlsolve(g!, [1.0; 1.1; 1.2; 1.15; 1.22; 1.21]) #solving for wages
    for i = 1:N*J
        wt0[i] = results.zero[i]
        wt[i, 1] = wt0[i]
    end
    println(wt0)  #end solving for wt0

    for time in 1:periods-1

        TSt1[:, :, time] .= [tradeSharest1(n, i, d1wdot, Ldot, Adot, kdot, time) for n in 1:N*J, i in 1:N]

        #############################################
        function f!(F, d1wdot)
            for n in 1:N*J
                j = div(n-1, N) + 1  # Find the correct index for j
                country = n % N 
                if country == 0
                    country = N
                end
                #F[n] = (d1wdot[n] * Ldot[n,time] * wt[n,time] * Lt[n,time] - sum(tradeSharest1((i-1) * J + j, country, d1wdot, Ldot, Adot, kdot, time)
                #    * Xt1((i-1) * J + j, d1wdot, tradesharest0, Ldot, Adot, kdot, time) for i in 1:N)) # Wrong indexing!!!
                F[n] = (incomet1(n, d1wdot, time) - sum(tradeSharest1((j-1) * N + i, country, d1wdot, Ldot, Adot, kdot, time)
                    * Xt1((j-1) * N + i, α, d1wdot, Lt, wt, Ldot, time) for i in 1:N)) 
            end
        end
        println("checkpoint")
        results = nlsolve(f!, [1.0; 1.1; 1.2; 1.15; 1.22; 1.21]) #solving for wages with country 1 set at 1.0
        sol = ones(N*J, periods)
        for i = 1:N*J
            sol[i, time] = results.zero[i] #saving results in sol[i]
            wdot[i, time] = sol[i,time]
            println(sol[i,time])
            wdot[i, time] = sol[i, time]
        end
        println("checkpoint")
        
        for n in 1:N
            pdotArray[n,time] = pdot(n, wdot, kdot, Adot,time)
        end
    
        tradesharest0 = TSt1[:, :, time] 
        

        wt[:, time + 1] .= wdot[:, time] .* wt[:, time]
    end

    for time in 1:periods-1
        for n in 1:N*J
            udotPathUpdate[n,time] = wdot[n,time]*(sum(μt[n,i,time]*(udotPathGuess[i,time+1])^(β/v) for i in N*J))^(v) ##needs verification but is equation 17 as specified by step 5
        end## Note: trying to figure out how to make sure the time+1 does not end the program with infs or NaN in the error
    end
    # Take the log difference of the guess and updated udots to get the error
    logudotPathGuess = log.(udotPathGuess) 

    logudotPathUpdate = log.(udotPathUpdate)
    logdifference = abs.(logudotPathGuess - logudotPathUpdate)
    errormax = maximum(logdifference[:,1:5])

    udotPathGuess = udotPathUpdate
    println("WHILE LOOP")
    display(errormax)

end

[1.137931563171395, 1.137931563171395, 1.175000000000299, 1.1750000000002987, 1.2150000000000118, 1.2150000000000116]
checkpoint
1.1059253158460292
1.1059253158460292
1.0710360199244016
1.0710360199244016
1.0357755748242623
1.0357755748242623
checkpoint
checkpoint
1.0135449387846642
1.0135449387846642
1.0135449387846631
1.0135449387846633
1.013544938784665
1.0135449387846647
checkpoint
checkpoint


NLsolve.IsFiniteException: During the resolution of the non-linear system, the evaluation of the following equation(s) resulted in a non-finite number: [1, 2, 3, 4, 5, 6]

In [9]:
#2/28/2025 with d1wdot instead of wdot
using Random, Distributions, Optim, NLsolve, SpecialFunctions
N = 2 # Number of countries
periods = 7
J = 3 # Number of goods (needs to be big number and an integer type) 
θ = fill(4.0, J) # Frechet shape parameter (governs comparative advantage)
T = ones(N*J, periods) # Frechet scale parameter (governs absolute advantage)
σ = 2 # Substitution elasticity between goods
τ = ones(N*J, N, periods) # Iceberg trade costs
L = ones(N*J, periods) # Size of labor force in each country
Ldot = ones(N*J, periods)


wt = ones(N*J, periods)
Lt = ones(N*J, periods)
wdot = ones(N*J, periods)
tradesharest0 = ones(N*J, N) * (1 / N)
Adot = ones(N*J, periods)

kdot = ones(N*J, N, periods)
TSt1 = ones(N*J, N, periods) 
pdotArray = ones(N*J, periods)
d1wdot = ones(N*J)
μ = zeros(N*J, N*J, periods)

μtminus1 = ones(N*J, N*J) #value of μ at time t = -1
πt0 = ones(N*J, N) #initial trade shares
wLt0 = ones(N*J, N) #initial values of w*L
rHt0 = ones(N*J, N) #initial values of r*H
μt = zeros(N*J,N*J,periods)

# a guess for path of udot:
udotPathGuess = ones(N*J, periods)
udotPathUpdate = zeros(N*J, periods)
errormax = 1.0 #the maximum log difference between guesses and updates for appendix D algorithm

function θindex(j)
    div(j-1, N) + 1  # Find the correct index for θ
end

function pdot(n, d1wdot, kdot, Adot, time)
    j = θindex(n)
    (sum(tradesharest0[n, i] * (d1wdot[(j-1) * N + i] * kdot[n, i, time])^-θ[j] * Adot[(j-1) * N + i, time]^θ[j] for i in 1:N))^(-1 / θ[j])
end ##### Going to need to update tradeshares

function tradeSharest1(n, i, d1wdot, Ldot, Adot, kdot, time)
    j = θindex(n)
    tradesharest0[n, i] * ((d1wdot[(j-1) * N + i] * kdot[n, i, time]) / pdot(n, d1wdot, kdot, Adot, time))^-θ[j] * Adot[(j-1) * N + i, time]^θ[j]
end
function incomet1(n, d1wdot, time)
    wt[n, time]*(d1wdot[n])*L[n, time]*(Ldot[n, time])
end
function Xt1(n, α, d1wdot, tradesharest0, Ldot, Adot, kdot, time)
    sum(incomet1(n, d1wdot, time) * tradeSharest1(n, i, d1wdot, Ldot, Adot, kdot, time) for i in 1:N)
end



while errormax > .01 
    for time in 1:periods-1
        for n in 1:N*J
            for i in 1:N*J
                if time == 1
                    μt[n,i,time] = μt[n,i,time] + (μtminus1[n,i]*udotPathGuess[i,time+1])/sum(μtminus1[n,i]*udotPathGuess[h,time+1] for h in 1:N*J)
                else 
                    μt[n,i,time] = μt[n,i,time] + (μt[n,i,time-1]*udotPathGuess[i,time+1])/sum(μt[n,h,time-1]*udotPathGuess[h,time+1] for h in 1:N*J)
                end
            end
        end
        for i in 1:N*J ### Step 3 of the appendix D
            for j in 1:N*J
                Lt[i ,time + 1] = Lt[i ,time + 1] + Lt[j, time]*μt[i,j,time]
            end
        end
    end

    for time in 1:periods-1
        for n in 1:N*J
            for i in 1:N
                TSt1[n, i, time] = tradeSharest1(n, i, d1wdot, Ldot, Adot, kdot, time)
            end
        end
        #############################################
        function f!(F, d1wdot)
            for n in 1:N*J
                j = div(n-1, N) + 1  # Find the correct index for j
                country = n % N 
                if country == 0
                    country = N
                end
                #F[n] = (d1wdot[n] * Ldot[n,time] * wt[n,time] * Lt[n,time] - sum(tradeSharest1((i-1) * J + j, country, d1wdot, Ldot, Adot, kdot, time)
                #    * Xt1((i-1) * J + j, d1wdot, tradesharest0, Ldot, Adot, kdot, time) for i in 1:N)) # Wrong indexing!!!
                F[n] = (incomet1(n, d1wdot, time) - sum(tradeSharest1((j-1) * N + i, country, d1wdot, Ldot, Adot, kdot, time)
                    * Xt1((j-1) * N + i, d1wdot, tradesharest0, Ldot, Adot, kdot, time) for i in 1:N))
            end
        end
        println("checkpoint")
        results = nlsolve(f!, [1.0; 1.1; 1.2; 1.15; 1.22; 1.21]) #solving for wages with country 1 set at 1.0
        sol = ones(N*J, periods)
        for i = 1:N*J
            sol[i, time] = results.zero[i] #saving results in sol[i]
            wdot[i, time] = sol[i,time]
            println(sol[i,time])
            wdot[i, time] = sol[i, time]
        end
        println("checkpoint")
        for n in 1:N
            pdotArray[n,time] = pdot(n, wdot, kdot, Adot,time)
        end
    
        for n in 1:N*J 
            for i in 1:N
                tradesharest0[n,i] = TSt1[n,i,time]
            end
        end
        for i in 1:N*J
            wt[i, time + 1] = wdot[i,time]*wt[i,time]
        end
    end

    for time in 1:periods-1
        for n in 1:N*J
            udotPathUpdate[n,time] = wt[n,time]*(sum(μt[n,i,time]*udotPathGuess[i,time+1] for i in N*J)) ##needs verification but is equation 17 as specified by step 5
        end ## Note: trying to figure out how to make sure the time+1 does not end the program with infs or NaN in the error
    end
    # Take the log difference of the guess and updated udots to get the error
    logudotPathGuess = log.(udotPathGuess) 

    logudotPathUpdate = log.(udotPathUpdate)
    logdifference = abs.(logudotPathGuess - logudotPathUpdate)
    errormax = maximum(logdifference[:,1:5])

    udotPathGuess = udotPathUpdate
    println("WHILE LOOP")
    display(errormax)

end

checkpoint
1.0666608752423143
1.0666608752423143
1.1880366027919622
1.1880366027919618
0.0011255377305359384
0.0011255377305359395
checkpoint
checkpoint
1.0629732231119513
1.0629732231119513
1.1894307508814022
1.1894307508814022
1.196910830060477
1.1969108300604772
checkpoint
checkpoint
1.0759222671742898
1.0759222671742898
1.193369855916085
1.1933698559160852
1.1716259959091488
1.1716259959091486
checkpoint
checkpoint
1.0648607335270892
1.0648607335270892
1.1970755682248726
1.1970755682248726
1.127135719853192
1.1271357198531922
checkpoint
checkpoint
1.0706428580335123
1.0706428580335121
1.1952904203340535
1.1952904203340535
1.1969288463747376
1.1969288463747374
checkpoint
checkpoint
1.0708151670507058
1.0708151670507058
1.1923240640716328
1.1923240640716328
1.2001760642465955
1.200176064246595
checkpoint
WHILE LOOP
checkpoint
1.0666608752423143
1.0666608752423143
1.1880366027919622
1.1880366027919618
0.0011255377305359384
0.0011255377305359395
checkpoint
checkpoint
1.0629732231119513

8.581253844052538

0.0

In [8]:

import Pkg; Pkg.add("SpecialFunctions")

   Resolving package versions...
    Updating `~/.julia/environments/v1.10/Project.toml`
  [276daf66] + SpecialFunctions v2.5.0
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
